## Задача 1

Реализовать класс для работы с линейной регрессией

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import time

class MyLinearRegression:
    def __init__(self,
                 regularization=None,
                 weight_calc='matrix',
                 lambda_1=None,
                 lambda_2=None,
                 batch_size=20,
                 max_iter=1000,
                 lr=0.001,
                 tol=1e-6):

        if regularization not in [None, 'l1', 'l2', 'l1l2']:
            raise TypeError(f"Параметр regularization не может принимать значение '{regularization}'")
        if weight_calc not in ['matrix', 'gd', 'sgd']:
            raise TypeError(f"Параметр weight_calc не может принимать значение '{weight_calc}'")
        if regularization in ['l1', 'l1l2'] and lambda_1 is None:
            raise TypeError("Значение коэффициента регуляризации l1 не задано")
        if regularization in ['l2', 'l1l2'] and lambda_2 is None:
            raise TypeError("Значение коэффициента регуляризации l2 не задано")
        if regularization in ['l1', 'l1l2'] and weight_calc == 'matrix':
            raise TypeError("При регуляризациях L1 или L1+L2 нельзя использовать weight_calc='matrix'")

        self.regularization = regularization
        self.weight_calc = weight_calc
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2
        self.batch_size = batch_size
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol

        self.coefs_ = None
        self.intercept_ = None

    def _add_intercept(self, X):
        return np.hstack([np.ones((X.shape[0], 1)), X])

    def _loss_and_grad(self, X, y, w):
        n = X.shape[0]
        y_pred = X @ w
        diff = y_pred - y.reshape(-1, 1)
        loss = (diff ** 2).mean()

        if self.regularization is not None:
            w_reg = w[1:]
            if self.regularization == 'l1':
                loss += self.lambda_1 * np.sum(np.abs(w_reg))  # L1 регуляризация
            elif self.regularization == 'l2':
                loss += self.lambda_2 * np.sum(w_reg**2)  # L2 регуляризация
            elif self.regularization == 'l1l2':
                loss += self.lambda_1 * np.sum(np.abs(w_reg)) + self.lambda_2 * np.sum(w_reg**2)

        grad = (2/n) * (X.T @ diff)

        # Регуляризация градиента
        if self.regularization is not None:
            w_reg = w[1:]
            if self.regularization in ['l2', 'l1l2']:
                grad[1:] += 2 * self.lambda_2 * w_reg  # Регуляризация L2
            if self.regularization in ['l1', 'l1l2']:
                grad[1:] += self.lambda_1 * np.sign(w_reg)  # Регуляризация L1

        return loss, grad

    # Метод для градиентного спуска (полный)
    def _fit_gd(self, X, y):
        w = np.random.randn(X.shape[1], 1) * 0.01
        prev_loss = np.inf

        for i in range(self.max_iter):
            loss, grad = self._loss_and_grad(X, y, w)
            w = w - self.lr * grad
            if abs(prev_loss - loss) < self.tol:
                break
            prev_loss = loss
        return w

    # Метод для стохастического градиентного спуска
    def _fit_sgd(self, X, y):
        n = X.shape[0]
        w = np.zeros((X.shape[1], 1))
        prev_loss = np.inf

        # Итерации стохастического градиентного спуска
        for i in range(self.max_iter):
            idx = np.random.randint(0, n, self.batch_size)
            X_batch = X[idx]
            y_batch = y[idx]
            loss, grad = self._loss_and_grad(X_batch, y_batch, w)
            w = w - self.lr * grad
            if abs(prev_loss - loss) < self.tol:
                break
            prev_loss = loss
        return w

    # Метод для решения задачи с помощью матричного метода
    def _fit_matrix(self, X, y):
        n, p = X.shape
        XTX = X.T @ X  # Матрица X^T * X
        XTy = X.T @ y.reshape(-1, 1)  # Вектор X^T * y

        # Решение для разных типов регуляризации
        if self.regularization is None:
            w = np.linalg.inv(XTX) @ XTy  # Решение без регуляризации
        elif self.regularization == 'l2':
            I = np.eye(p)  # Единичная матрица для регуляризации
            I[0, 0] = 0
            w = np.linalg.inv(XTX + self.lambda_2 * I) @ XTy  # Регуляризация L2
        else:
            raise ValueError("Неподдерживаемое сочетание для матричного решения")
        return w

    # Основной метод для обучения модели
    def fit(self, X: pd.DataFrame, y: pd.DataFrame):
        X_ext = self._add_intercept(X)
        if self.weight_calc == 'matrix':
            w = self._fit_matrix(X_ext, y)
        elif self.weight_calc == 'gd':
            w = self._fit_gd(X_ext, y)
        else:
            w = self._fit_sgd(X_ext, y)

        self.intercept_ = w[0, 0]
        self.coefs_ = w[1:].ravel()
        return self

    # Метод для предсказания на новых данных
    def predict(self, X: np.array):
        X_ext = self._add_intercept(X)
        return X_ext @ np.concatenate([[self.intercept_], self.coefs_]).reshape(-1, 1)

    # Метод для оценки точности модели (R^2)
    def score(self, X: np.array, y: np.array):
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred.ravel())**2)  # Остаточная сумма квадратов
        ss_tot = np.sum((y - y.mean())**2)  # Общая сумма квадратов
        r2 = 1 - ss_res / ss_tot  # Рассчитываем коэффициент детерминации (R^2)
        return r2

In [ ]:
data = pd.read_csv("/content/data.csv")

# Целевой признак
y = data['price'].values

# Признаки
X = data.drop(columns='price')

# Нормализация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.drop(columns=['model', 'transmission']))

# Кодируем категориальные признаки
cat_cols = ['model', 'transmission']
enc = OneHotEncoder(sparse_output=False, drop='first')
X_cat = enc.fit_transform(X[cat_cols])

X_all = np.hstack([X_cat, X_scaled])

X_train, y_train = X_all, y
X_test, y_test = X_all, y

# Сравнение различных конфигураций
configs = [
    (None, 'matrix', None, None),
    ('l2', 'matrix', None, 10.0),
    ('l2', 'gd', None, 10.0),
    ('l1', 'gd', 1.0, None),
    ('l1l2', 'sgd', 1.0, 10.0),
]

for regularization, weight_calc, l1, l2 in configs:
    model = MyLinearRegression(
        regularization=regularization,
        weight_calc=weight_calc,
        lambda_1=l1,
        lambda_2=l2,
        max_iter=100,
        lr=0.0001
    )

    start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start

    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    prediction_time_start = time.time()
    y_pred = model.predict(X_test)
    prediction_time = time.time() - prediction_time_start

    print(f"regularization={regularization}, weight_calc={weight_calc}, l1={l1}, l2={l2}")
    print(f"Intercept: {model.intercept_}")
    print(f"Train R^2: {train_score}, Test R^2: {test_score}")
    print(f"Training time: {train_time:.6f}s, Prediction time: {prediction_time:.6f}s")
    print("-"*50)

regularization=None, weight_calc=matrix, l1=None, l2=None
Intercept: 5987.703609416494
Train R^2: 0.8534638500411593, Test R^2: 0.8534638500411593
Training time: 0.000245s, Prediction time: 0.000038s
--------------------------------------------------
regularization=l2, weight_calc=matrix, l1=None, l2=1.0
Intercept: 5983.364089389988
Train R^2: 0.8534018898281588, Test R^2: 0.8534018898281588
Training time: 0.000224s, Prediction time: 0.000038s
--------------------------------------------------
regularization=l2, weight_calc=gd, l1=None, l2=1.0
Intercept: 5983.239823211249
Train R^2: 0.8533677843956781, Test R^2: 0.8533677843956781
Training time: 0.005685s, Prediction time: 0.000039s
--------------------------------------------------
regularization=l1, weight_calc=gd, l1=0.1, l2=None
Intercept: 5983.512307128194
Train R^2: 0.8533592278362112, Test R^2: 0.8533592278362112
Training time: 0.006075s, Prediction time: 0.000040s
--------------------------------------------------
regularizatio

Используя датасет про автомобили (целевой признак — price), сравнить (качество, скорость обучения и предсказания, важность признаков) модели `MyLinearRegression` с различными гиперпараметрами, сделать выводы.

Время обучения и предсказания незначительно влияет на качество модели, которое везде остаётся на уровне около 0.853 (практически идентично). Это указывает на то, что изменения гиперпараметров, связанные с регуляризацией и методами расчёта весов, не оказывают значительного влияния на точность предсказания.

Модели с регуляризацией L2 и L1 имеют схожие значения по времени, но использование комбинированной регуляризации (L1L2) увеличивает время обучения и предсказания, что может быть связано с дополнительной сложностью модели.

Включение регуляризации (L1 или L2) в целом не ведёт к значительному ухудшению качества модели. Однако, комбинация L1 и L2 регуляризаций немного снижает R² и увеличивает время работы модели, что нужно учитывать при выборе гиперпараметров в реальных приложениях.

Таким образом, для данной задачи можно рекомендовать использование модели с L2 регуляризацией без значительного увеличения времени работы, если дополнительная сложность, связанная с L1 или комбинированной регуляризацией, не приносит значимого улучшения в результатах.

## Задача 2

[Соревнование на Kaggle](https://www.kaggle.com/competitions/stat-plus-ml-2024)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Загрузка данных
train_data = pd.read_csv('train_contest.csv')
test_data = pd.read_csv('for_prediction.csv')

train_data['area'] = train_data['area'].apply(lambda x: eval(x)['name'] if isinstance(eval(x), dict) else x)
test_data['area'] = test_data['area'].apply(lambda x: eval(x)['name'] if isinstance(eval(x), dict) else x)

train_data['experience'] = train_data['experience'].apply(lambda x: eval(x)['name'] if isinstance(eval(x), dict) else x)
test_data['experience'] = test_data['experience'].apply(lambda x: eval(x)['name'] if isinstance(eval(x), dict) else x)

preprocessor = ColumnTransformer(
    transformers=[
        ('area', OneHotEncoder(handle_unknown='ignore'), ['area']),
        ('experience', OneHotEncoder(handle_unknown='ignore'), ['experience']),
        ('premium', SimpleImputer(strategy='mean'), ['premium'])
    ])

# Признаки для обучения
X = train_data[['area', 'premium', 'experience']]
y = train_data['mean_salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Обучение модели
model.fit(X_train, y_train)

# Прогнозирование на тестовых данных
y_pred = model.predict(X_test)

# Оценка модели (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

# Прогнозирование на новых данных
X_new = test_data[['area', 'premium', 'experience']]  # Используем те же признаки для тестовых данных
y_new_pred = model.predict(X_new)

# Формирование DataFrame с результатами для отправки
submission = pd.DataFrame({'Id': test_data['Id'], 'Predicted': y_new_pred})
submission.to_csv('submission.csv', index=False)

print("Прогнозы сохранены в 'submission.csv'")


Mean Absolute Error (MAE): 50003.02719272275
Прогнозы сохранены в 'submission.csv'
